In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/A2Summ

/content/drive/MyDrive/A2Summ


### Generate Captions and save them

In [ ]:
import os
import json
import h5py
import cv2
import numpy as np
from tqdm import tqdm
import torch
from PIL import Image
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer

In [ ]:
# 1. Load Video-to-Name Mapping
# Load the provided mapping file
with open('data/TVSum/mapped_video_names.json', 'r') as f:
    video_name_mapping = json.load(f)

print(f"Loaded mapping for {len(video_name_mapping)} videos")
print("First 3 mappings:")
for i, (video_id, video_name) in enumerate(list(video_name_mapping.items())[:3]):
    print(f"  {video_id} -> {video_name}")

Loaded mapping for 50 videos
First 3 mappings:
  AwmHb44_ouw -> video_1
  98MoyGZKHXc -> video_2
  J0nA4VgnoCo -> video_3


In [ ]:
# 2. Load the Image Captioning Model
# Load the vit-gpt2-image-captioning model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

model.to(device)

# Configure generation parameters
max_length = 32
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

# 3. Define Helper Functions
def predict_caption(image):
    """
    Generate a caption for the given image using vit-gpt2-image-captioning model
    """
    if isinstance(image, np.ndarray):
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    pixel_values = feature_extractor(images=[image], return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model.generate(pixel_values, **gen_kwargs)
    caption = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]
    return caption

def extract_frame(video_path, frame_number):
    """
    Extract a specific frame from a video using OpenCV
    """
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    cap.release()
    if ret:
        return frame
    return None

# 4. Set Up Output Directory
# Create output directories
output_dir = "data/TVSum_captions"
frames_dir = os.path.join(output_dir, "frames")
os.makedirs(output_dir, exist_ok=True)
os.makedirs(frames_dir, exist_ok=True)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/982M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [ ]:
# 5. Extract and Caption Frames from TVSum Dataset
# Define path to the h5 file containing TVSum dataset information
dataset_name = "TVSum"
data_root = "."
h5_file_path = "/content/drive/MyDrive/A2Summ/data/TVSum/feature/eccv16_dataset_tvsum_google_pool5.h5"

# Dictionary to store all captions
all_captions = {}

try:
    # Open the h5 file containing the TVSum dataset
    with h5py.File(h5_file_path, 'r') as h5_file:
        # Process each video
        for mapped_name, video_id in tqdm(video_name_mapping.items(), desc="Processing videos"):
            video_path = os.path.join("/content/drive/MyDrive/A2Summ/ydata-tvsum50-v1_1/video", f"{mapped_name}.mp4")

            # Skip if video file doesn't exist
            if not os.path.exists(video_path):
                print(f"Warning: Video file not found: {video_path}")
                continue

            # Check if video exists in h5 file
            if video_id not in h5_file:
                print(f"Warning: Video ID {video_id} not found in h5 file")
                continue

            # Get video data from h5 file
            video_data = h5_file[video_id]

            # Get the 'picks' (frame indices to extract)
            if 'picks' not in video_data:
                print(f"Warning: No 'picks' found for video {video_id}")
                continue

            picks = video_data['picks'][...].astype(np.int32)
            n_frames = video_data['n_frames'][...].astype(np.int32) if 'n_frames' in video_data else None

            print(f"Processing {mapped_name} (ID: {video_id}) with {len(picks)} frames")

            # Create directory for this video's frames
            video_frames_dir = os.path.join(frames_dir, mapped_name)
            os.makedirs(video_frames_dir, exist_ok=True)

            # Dictionary to store captions for this video
            video_captions = {}

            # Process each frame in the 'picks' list
            for i, frame_idx in enumerate(tqdm(picks, desc=f"Captioning frames for {mapped_name}")):
                # Extract frame from video
                frame = extract_frame(video_path, frame_idx)

                if frame is None:
                    print(f"Warning: Could not extract frame {frame_idx} from {video_id}")
                    continue

                # Save frame as image
                frame_path = os.path.join(video_frames_dir, f"{frame_idx}.jpg")
                cv2.imwrite(frame_path, frame)

                # Generate caption for frame
                caption = predict_caption(frame)
                video_captions[str(int(frame_idx))] = caption

                # Print progress occasionally
                if (i + 1) % 10 == 0 or i == 0 or i == len(picks) - 1:
                    print(f"  Frame {frame_idx}: '{caption}'")

            # Store captions for this video
            all_captions[video_id] = {
                "name": mapped_name,
                "frame_captions": video_captions,
                "picks": picks.tolist()
            }

            # Save progress after each video
            with open(os.path.join(output_dir, "captions.json"), 'w') as f:
                json.dump(all_captions, f, indent=2)

            print(f"Completed captioning {len(video_captions)} frames for {mapped_name}")

    print("All videos processed successfully!")

except Exception as e:
    print(f"Error: {e}")
    # Save partial results if there was an error
    if all_captions:
        with open(os.path.join(output_dir, "captions_partial.json"), 'w') as f:
            json.dump(all_captions, f, indent=2)
        print("Saved partial results")

Processing videos:   0%|          | 0/50 [00:00<?, ?it/s]

Processing AwmHb44_ouw (ID: video_1) with 707 frames



Captioning frames for AwmHb44_ouw:   0%|          | 0/707 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.

Captioning frames for AwmHb44_ouw:   0%|          | 1/707 [00:04<55:23,  4.71s/it]

  Frame 0: 'a yellow and black tire sitting next to a black car '



Captioning frames for AwmHb44_ouw:   1%|▏         | 10/707 [00:12<10:27,  1.11it/s]

...

  Frame 6735: 'a cat sitting on the floor in front of a door '



Captioning frames for -esJrBWj2d8: 100%|█████████▉| 460/461 [04:02<00:00,  1.89it/s]

  Frame 6885: 'a cat sitting on the floor next to a door '



Processing videos: 100%|██████████| 50/50 [3:08:53<00:00, 226.68s/it]

  Frame 6900: 'a cat sitting on the floor next to a door '
Completed captioning 461 frames for -esJrBWj2d8
All videos processed successfully!


In [ ]:
# 6. Generate Summary Statistics
# Generate summary statistics for captioning results
try:
    with open(os.path.join(output_dir, "captions.json"), 'r') as f:
        all_captions = json.load(f)

    # Calculate statistics
    total_videos = len(all_captions)
    total_frames = sum(len(video_data["frame_captions"]) for video_data in all_captions.values())

    print("Captioning Summary:")
    print(f"  Total videos processed: {total_videos}")
    print(f"  Total frames captioned: {total_frames}")
    print(f"  Average frames per video: {total_frames / max(1, total_videos):.1f}")

    # Show sample captions
    print("\nSample Captions:")
    for video_id, video_data in list(all_captions.items())[:3]:
        video_name = video_data["name"]
        frame_captions = video_data["frame_captions"]
        if frame_captions:
            # Take a sample from the middle
            frame_keys = list(frame_captions.keys())
            sample_frame = frame_keys[len(frame_keys) // 2]
            print(f"  {video_name}, Frame {sample_frame}: '{frame_captions[sample_frame]}'")
except Exception as e:
    print(f"Error generating summary: {e}")

Captioning Summary:
  Total videos processed: 50
  Total frames captioned: 23510
  Average frames per video: 470.2

Sample Captions:
  AwmHb44_ouw, Frame 5295: 'a person's hand holding a tire in the dirt '
  98MoyGZKHXc, Frame 2340: 'a man sitting on the hood of a parked car '
  J0nA4VgnoCo, Frame 7005: 'a newspaper with a picture of an airplane on it '


### Filter positive frame captions

In [ ]:
import os
import json
import numpy as np
from tqdm import tqdm

# Paths to input and output directories
frames_data_folder = "data/TVSum_identified_key_frames"  # Contains model output and ground truth frame indices
captions_folder = "data/TVSum_captions"   # Contains the already generated captions
output_folder = captions_folder      # Save the organized results here

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Load the captions
try:
    with open(os.path.join(captions_folder, "captions.json"), "r") as f:
        all_captions = json.load(f)
    print(f"Loaded captions for {len(all_captions)} videos")
except Exception as e:
    print(f"Error loading captions: {e}")
    exit(1)

# Initialize result dictionaries
a2summ_results = {}
gt_results = {}

# Get the list of JSON files in the frames folder
frame_files = [f for f in os.listdir(frames_data_folder) if f.endswith('.json')]
print(f"Found {len(frame_files)} videos with frame data")

# Process each video
for frame_file in tqdm(frame_files, desc="Organizing frames"):
    # Load the frame data (model output and ground truth)
    with open(os.path.join(frames_data_folder, frame_file), 'r') as f:
        frame_data = json.load(f)

    video_name = frame_data['video_name']
    identified_key_frames = np.array(frame_data['identified_key_frames'])  # Model's identified frames
    gt_key_frames = np.array(frame_data['gt_key_frames'])  # Ground truth frames

    # Find corresponding video ID in the captions data
    # corresponding_video_id = None
    # for video_id, video_data in all_captions.items():
    #     if video_data["name"] == video_name:
    #         corresponding_video_id = video_id
    #         break

    # if not corresponding_video_id:
    #     print(f"Warning: Couldn't find captions for {video_name}")
    #     continue
    corresponding_video_id = video_name

    # Get captions and picks for this video
    video_captions = all_captions[corresponding_video_id]["frame_captions"]
    picks = np.array(all_captions[corresponding_video_id]["picks"])

    print(f"Processing {video_name} with {len(picks)} frames")

    # Initialize results for this video
    a2summ_video_results = {"positive_frames": {}, "negative_frames": {}}
    gt_video_results = {"positive_frames": {}, "negative_frames": {}}

    # Maps to track which picks are marked as positive
    a2summ_positive_picks = set()
    gt_positive_picks = set()

    # Find picks that are close to identified key frames
    for key_frame in identified_key_frames:
        # Find the closest pick to this key frame
        distances = np.abs(picks - key_frame)
        closest_pick_idx = distances.argmin()

        # If the distance is within a threshold, mark it as positive
        if distances[closest_pick_idx] <= 15:  # Threshold of 15 frames (about 0.5s at 30fps)
            a2summ_positive_picks.add(closest_pick_idx)

    # Find picks that are close to ground truth key frames
    for key_frame in gt_key_frames:
        # Find the closest pick to this key frame
        distances = np.abs(picks - key_frame)
        closest_pick_idx = distances.argmin()

        # If the distance is within a threshold, mark it as positive
        if distances[closest_pick_idx] <= 15:  # Same threshold
            gt_positive_picks.add(closest_pick_idx)

    # Categorize each pick as positive or negative
    for i, pick in enumerate(picks):
        pick_str = str(int(pick))

        # Skip if caption doesn't exist for this frame
        if pick_str not in video_captions:
            continue

        caption = video_captions[pick_str]

        # Categorize for A2Summ
        if i in a2summ_positive_picks:
            a2summ_video_results["positive_frames"][pick_str] = caption
        else:
            a2summ_video_results["negative_frames"][pick_str] = caption

        # Categorize for ground truth
        if i in gt_positive_picks:
            gt_video_results["positive_frames"][pick_str] = caption
        else:
            gt_video_results["negative_frames"][pick_str] = caption

    # Store results for this video
    a2summ_results[video_name] = a2summ_video_results
    gt_results[video_name] = gt_video_results

    # Print some statistics
    print(f"  A2Summ - Positive: {len(a2summ_video_results['positive_frames'])}, " +
          f"Negative: {len(a2summ_video_results['negative_frames'])}")
    print(f"  GT     - Positive: {len(gt_video_results['positive_frames'])}, " +
          f"Negative: {len(gt_video_results['negative_frames'])}")

# Save the final results
with open(os.path.join(output_folder, "a2summ_captions.json"), 'w') as f:
    json.dump(a2summ_results, f, indent=2)
print(f"Saved A2Summ results to {os.path.join(output_folder, 'a2summ_captions.json')}")

with open(os.path.join(output_folder, "gt_captions.json"), 'w') as f:
    json.dump(gt_results, f, indent=2)
print(f"Saved ground truth results to {os.path.join(output_folder, 'gt_captions.json')}")

print("Processing complete!")

Loaded captions for 50 videos
Found 28 videos with frame data


Organizing frames:   7%|▋         | 2/28 [00:00<00:01, 16.70it/s]

Processing video_33 with 891 frames
  A2Summ - Positive: 152, Negative: 739
  GT     - Positive: 144, Negative: 747
Processing video_21 with 1294 frames
  A2Summ - Positive: 220, Negative: 1074
  GT     - Positive: 213, Negative: 1081
Processing video_43 with 329 frames
  A2Summ - Positive: 55, Negative: 274
  GT     - Positive: 53, Negative: 276
Processing video_31 with 361 frames
  A2Summ - Positive: 49, Negative: 312
  GT     - Positive: 47, Negative: 314
Processing video_27 with 728 frames
  A2Summ - Positive: 50, Negative: 678
  GT     - Positive: 47, Negative: 681
Processing video_1 with 707 frames


Organizing frames:  43%|████▎     | 12/28 [00:00<00:00, 40.68it/s]

  A2Summ - Positive: 121, Negative: 586
  GT     - Positive: 118, Negative: 589
Processing video_34 with 247 frames
  A2Summ - Positive: 41, Negative: 206
  GT     - Positive: 41, Negative: 206
Processing video_38 with 197 frames
  A2Summ - Positive: 25, Negative: 172
  GT     - Positive: 19, Negative: 178
Processing video_10 with 267 frames
  A2Summ - Positive: 45, Negative: 222
  GT     - Positive: 44, Negative: 223
Processing video_23 with 376 frames
  A2Summ - Positive: 64, Negative: 312
  GT     - Positive: 62, Negative: 314
Processing video_24 with 291 frames
  A2Summ - Positive: 50, Negative: 241
  GT     - Positive: 48, Negative: 243
Processing video_35 with 298 frames
  A2Summ - Positive: 54, Negative: 244
  GT     - Positive: 47, Negative: 251
Processing video_50 with 461 frames
  A2Summ - Positive: 78, Negative: 383
  GT     - Positive: 77, Negative: 384
Processing video_16 with 636 frames
  A2Summ - Positive: 111, Negative: 525
  GT     - Positive: 107, Negative: 529
Proces

Organizing frames:  79%|███████▊  | 22/28 [00:00<00:00, 44.55it/s]

Processing video_26 with 221 frames
  A2Summ - Positive: 34, Negative: 187
  GT     - Positive: 36, Negative: 185
Processing video_41 with 539 frames
  A2Summ - Positive: 85, Negative: 454
  GT     - Positive: 88, Negative: 451
Processing video_36 with 531 frames
  A2Summ - Positive: 90, Negative: 441
  GT     - Positive: 84, Negative: 447
Processing video_42 with 396 frames
  A2Summ - Positive: 68, Negative: 328
  GT     - Positive: 66, Negative: 330
Processing video_40 with 761 frames
  A2Summ - Positive: 75, Negative: 686
  GT     - Positive: 73, Negative: 688
Processing video_46 with 1021 frames
  A2Summ - Positive: 46, Negative: 975
  GT     - Positive: 46, Negative: 975
Processing video_18 with 649 frames
  A2Summ - Positive: 115, Negative: 534
  GT     - Positive: 113, Negative: 536
Processing video_44 with 287 frames
  A2Summ - Positive: 45, Negative: 242
  GT     - Positive: 45, Negative: 242
Processing video_15 with 289 frames
  A2Summ - Positive: 43, Negative: 246
  GT     -

Organizing frames: 100%|██████████| 28/28 [00:00<00:00, 37.42it/s]

  A2Summ - Positive: 192, Negative: 977
  GT     - Positive: 194, Negative: 975
Processing video_22 with 378 frames
  A2Summ - Positive: 67, Negative: 311
  GT     - Positive: 66, Negative: 312
Processing video_28 with 553 frames
  A2Summ - Positive: 93, Negative: 460
  GT     - Positive: 93, Negative: 460
Saved A2Summ results to data/TVSum_captions/a2summ_captions.json
Saved ground truth results to data/TVSum_captions/gt_captions.json
Processing complete!


In [ ]:
import h5py
import numpy as np

h5_file_path = "/content/drive/MyDrive/A2Summ/data/TVSum/feature/eccv16_dataset_tvsum_google_pool5.h5"

with h5py.File(h5_file_path, 'r') as h5_file:
  video_data = h5_file["video_33"]

  for i in video_data.keys():
    try:
      print(i, video_data[i].shape)
      print(i, video_data[i][0])
    except:
      print(";;", i, video_data[i][...])

    user1_summ = video_data["user_summary"][0]
    print(np.sum(user1_summ))

change_points (90, 2)
change_points [ 0 30]
1994.0
features (891, 1024)
features [0.00025534 0.00713549 0.0009231  ... 0.         0.00108624 0.00038016]
1994.0
gtscore (891,)
gtscore 0.0875
1994.0
gtsummary (891,)
gtsummary 0.0
1994.0
n_frame_per_seg (90,)
n_frame_per_seg 31
1994.0
n_frames ()
;; n_frames 13365
1994.0
n_steps ()
;; n_steps 891
1994.0
picks (891,)
picks 0
1994.0
user_summary (20, 13365)
user_summary [1. 1. 1. ... 1. 1. 1.]
1994.0
